In [3]:
import pandas as pd
import numpy as np

import os
import re
import json
from itertools import chain
from time import strftime, localtime
os.chdir('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive')
from NLP_function import *
os.chdir('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/Run')

import pickle
import jieba

import gensim
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from keras.preprocessing.text import text_to_word_sequence
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()

##### * parameters *

In [4]:
file = 'C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/Data_Prepare/Project_pogress_20180509.xlsx'
#sampid = ['A2172','A4305','A2769','A2023', 'A2042','A2108',
#          'A3455','A3450','A3439','A3435','A3444','A4304','A3691','A32940','A2023']
sampid = ['A4537','A4222','A4382','A4442','A3581','A4583']
#sampid = ['A3324']
topn = 10 # choose top n terms to show (doc2vec)

#### Data import _ use original clinical data

In [5]:
# use original clinical data
df = pd.read_excel(file)
df = df.iloc[:, np.r_[0, 5, 17, 21:58]]
df = df.loc[df['样本编号'].isin(sampid)]

# merge title and content
for a in range(0,df.shape[1]-1):
    if not a in np.r_[0,1,2,3,7:19,22:df.shape[1]]:
        i = df.columns[a]
        df[i] = df[i].apply(lambda x : str(x) if pd.isnull(x) else i.replace('精阅体格检查\n/','') + str(x) )
        df[i] = df[i].apply(lambda x : x.replace('伊阅',''))  
        df[i] = df[i].apply(lambda x : x.replace('nan', ''))  
        #df = df[~df[i].isin([''])]

# transform into list and remove float phenotype
df = df.T
df.columns = df.iloc[0]
res = []
for i in range(0,df.shape[1]):
    res.append(pd.Series(df.iloc[0:,i]).dropna().drop_duplicates().values) 
for i in range(0,len(res)):
    res[i] = [x for x in res[i] if type(x) != float] 


# tokenization
for a in range (0,len(res)):
    for b in range(0,len(res[a])):
        res[a][b] = re.split('[，]|[,]|[；]|[：]', res[a][b])
        
# save and export to numpy array
#np.save('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive/Run/cli_tokenizing_res.npy', np.array(res))

# read hpo data
hpo = pd.read_csv('C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/hpo_import_python/Hpo_cn_clean.csv')

#### Prediction

In [13]:
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load("C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/Not_In_Database/unsup/M_Doc2Vec_BOW.model")

from translate import Translator
translator = Translator(from_lang='zh', to_lang='en')

#### tokenization (top 10 most similar docs + top 3 terms search in hpo)

In [14]:
### extract information
txt = pd.DataFrame(columns=['id', 'pheno'])
for i in range(0,len(res)):
    length = len(list(chain(*res[i][2:len(res[0])])) )
    tem = pd.DataFrame({'id': pd.DataFrame(res[i][0]).values.repeat(length), 'pheno': list(chain(*res[i][2:len(res[0])]))  })
    txt = txt.append(tem)
txt['pheno'] = txt['pheno'].dropna()
txt = txt.reset_index(drop=True)

### clean txt (remove non sense doc)
searchfor = ["正常"]

txt['pheno'] = txt['pheno'].apply(clean_text, lan='cn').dropna()

txt = txt[~ txt.pheno.str.contains('|'.join(searchfor), na=False)]
txt = txt[~ txt.pheno.isin(['', '.', '重点关注','无','全外扫描','未查','B超','其他遗传学检查结果','其他检查','要求全外扫描'])] # remove str contraing specific str
txt['pheno'] = txt['pheno'].apply(lambda x: x.replace('45,X','45_X'))

txt = txt.reset_index(drop=True)

### find most similar doc using tags
txt['pheno_tok'] = txt['pheno_en_tok'] = txt['sim'] = txt['vec'] = txt['pheno_en'] = txt['pheno']

for p in range(len(txt)):
    txt['pheno_en'][p] = translator.translate(txt['pheno'][p])
    txt['pheno_en_tok'][p] = text_to_word_sequence(txt['pheno_en'][p])
    txt['vec'][p] = word_tokenize(txt['pheno_en'][p].lower())
    txt['vec'][p] = model.infer_vector(txt['vec'][p])
    txt['sim'][p] = model.docvecs.most_similar([txt['vec'][p]], topn = topn)

txt['pheno_tok'] = txt['pheno'].apply(lambda x: list(jieba.cut(x)))    
#txt.head()

RuntimeError: generator raised StopIteration

<bound method Translator.translate of <translate.translate.Translator object at 0x000001403946BB50>>

#### save results

In [8]:
txt['doc2vec']  = np.empty((len(txt), 0)).tolist()
txt['sent_search']  = np.empty((len(txt), 0)).tolist()
txt['word_search']  = np.empty((len(txt), 0)).tolist()
txt['jieba_search']  = np.empty((len(txt), 0)).tolist()
txt['common_term'] = np.empty((len(txt), 0)).tolist()


for a in range(0,len(txt['sim'])):
    
    
# doc2vec search
    for b in range(0,len(txt['sim'][a])-1):
        index = txt['sim'][a][b][0]
        txt['doc2vec'][a].append(str(list(hpo.iloc[:,[0,2]].loc[int(index)]))[1:-1])

        
# sentence search        
    if(len(txt['pheno'][a])>0): # or >1
        sent = txt['pheno'][a]  # possible to use n-gram
        if (len(hpo[hpo['merge_cn'].str.contains(sent, na=False)])>0):
            res = hpo[hpo['merge_cn'].str.contains(sent, na=False)].iloc[:,[0,2]]
            if (len(res)<=3): l=len(res)
            else: l=3
            for c in range(l):
                txt['sent_search'][a].append(str(list(res.iloc[c]))[1:-1]) 
  

# word search (english word tokens)      
    if(len(text_to_word_sequence(txt['pheno_en'][a]))>0): # or >1
        words = text_to_word_sequence(txt['pheno_en'][a])  # possible to use n-gram
        for word in words:
            word = lemmatizer.lemmatize(word)
            word = re.compile('[0-9]').sub('',word) # remove numbers
            word = clean_text(word, lan='en')        # remove stop words                
            # word = PorterStemmer().stem(word)
            if (len(clean_text(word, lan='en'))>0): 
                if (len(hpo[hpo['merge'].str.contains(word, na=False)])>0): 
                    #txt['word_search'][a].append('**'+word +'**')
                    res = hpo[hpo['merge'].str.contains(word, na=False)].iloc[:,[0,2]]
                    if (len(res)<=3): l=len(res)
                    else: l=3
                    for c in range(l):
                        txt['word_search'][a].append(str(list(res.iloc[c]))[1:-1])
    
    
    # jieba search (chinese tokens)   
    if(len(txt['pheno_tok'][a])>0): # or >1
        words_z = txt['pheno_tok'][a]  # possible to use n-gram
        for word in words_z:
            if (len(word)>0):
                if (len(hpo[hpo['merge_cn'].str.contains(word, na=False)])>0): 
                    res = hpo[hpo['merge_cn'].str.contains(word, na=False)].iloc[:,[0,2]]
                    if (len(res)<=3): l=len(res)
                    else: l=3
                    for c in range(l):
                        txt['jieba_search'][a].append(str(list(res.iloc[c]))[1:-1]) 
                        
                        

# search common terms (no repeat)
    txt['common_term'][a] = list(set(list(set(txt['jieba_search'][a])&set(txt['word_search'][a])) + 
                                     list(set(txt['doc2vec'][a])&set(txt['word_search'][a])) + 
                                     list(set(txt['jieba_search'][a] )&set(txt['doc2vec'][a])) +
                                     list(set(txt['sent_search'][a])&set(txt['doc2vec'][a])) +
                                     list(set(txt['sent_search'][a])&set(txt['jieba_search'][a])) + 
                                     list(set(txt['sent_search'][a])&set(txt['word_search'][a]))                                     
                                    ))       

In [8]:
# save 
res = txt[['id','pheno','pheno_en','common_term','sent_search','doc2vec', 'jieba_search','word_search']]
res.to_csv('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive/Run/prediction_unsp_'+ strftime("%Y%m%d_%H%M", localtime()) +'.csv', header=True, index=False, encoding="ANSI")